- torch molecule つかう

In [1]:
import os

is_kaggle_notebook = os.path.exists("/kaggle/input")

# 必要パッケージをインストール
if is_kaggle_notebook:
    !pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
    !pip install /kaggle/input/torch-geometric-2-6-1/torch_geometric-2.6.1-py3-none-any.whl

In [2]:
import os
import warnings
import sys
from pathlib import Path

import lightgbm as lgb
import numpy as np
import pandas as pd
from rdkit import rdBase

import torch
# from torch.utils.data import Subset
# from torch_geometric.loader import DataLoader
from torch_molecule import GNNMolecularPredictor


from sklearn.model_selection import KFold, GroupKFold
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error


rdBase.DisableLog('rdApp.warning')
warnings.filterwarnings("ignore")

In [3]:
pr_number = 1

In [4]:
if is_kaggle_notebook:
    module_path = f"/kaggle/input/myproject-pr-{pr_number:04}"
    !mkdir src
    !cp -r $module_path/* src/
    src_path = "./"
else:
    src_path = "../"

sys.path.append(src_path)

from src.data import load_data, add_descriptors, add_external_data
# from src.model import MoleculeDataset, GNNModel, smiles_to_graph, ATOM_MAP
from src.utils import score, NULL_FOR_SUBMISSION, generate_scaffold


In [32]:
exp = "exp016"
debug = False

if is_kaggle_notebook:
    debug = False

n_splits = 5 if not debug else 2
targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg'] if not debug else ["Tg"]

num_epochs = 10 if not debug else 10
batch_size = 128
hidden_channels = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# ---------------------------
# メイン処理
# ---------------------------
if debug:
    output_path = Path("../tmp") / exp
else:
    output_path = Path("../outputs") / exp

if is_kaggle_notebook:
    # kaggle notebook
    data_dir = Path("/kaggle/input")
else:
    # local
    data_dir = Path("../data/raw")

train, test = load_data(data_dir)
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv", "Tg")
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv", "Tc", {"TC_mean": "Tc"})
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv", "FFV")


In [85]:
# ---------------------------
# メイン処理
# ---------------------------
if debug:
    output_path = Path("../tmp") / exp
else:
    output_path = Path("../outputs") / exp

if is_kaggle_notebook:
    # kaggle notebook
    data_dir = Path("/kaggle/input")
else:
    # local
    data_dir = Path("../data/raw")

train, test = load_data(data_dir)
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv", "Tg")
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv", "Tc", {"TC_mean": "Tc"})
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv", "FFV")

train["id"] = np.arange(len(train))
train["scaffold"] = train["SMILES"].apply(generate_scaffold)
submission = pd.read_csv(data_dir / "neurips-open-polymer-prediction-2025/sample_submission.csv")

oof_dfs = []

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
# gkf = GroupKFold(n_splits=n_splits)
X = train[["SMILES"]]
y = train[targets]

X_test = test["SMILES"].to_list()
groups = train["scaffold"]
gkf.get_n_splits(X, y, groups)
fold_results = []
oof = np.zeros((len(train), len(targets)))


# for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n=== Fold {fold + 1} / {n_splits} ===")
    
    X_train = X.iloc[train_idx]["SMILES"].to_list()
    y_train = y.iloc[train_idx][targets].to_numpy()
    X_val = X.iloc[val_idx]["SMILES"].to_list()
    y_val = y.iloc[val_idx][targets].to_numpy()
    
    gnn = GNNMolecularPredictor(
        task_type="regression",
        num_task=len(targets),
        batch_size=batch_size,
        epochs=num_epochs,
        verbose=True
    )
    gnn.autofit(
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        n_trials=1,
    )
    preds = gnn.predict(X_val)["prediction"]
    sub = gnn.predict(X_test)["prediction"]
    
    trues = y_val
    
    oof[val_idx] = preds

    for idx, target in enumerate(targets):
        submission[target] = sub[:, idx]
        
        y_true = trues[:, idx]
        y_pred = preds[:, idx]
        
        cond = ~np.isnan(y_true)
        mse = mean_squared_error(y_true[cond], y_pred[cond])
        mae = mean_absolute_error(y_true[cond], y_pred[cond])
        print(f"Fold {fold+1} target: {target} MSE: {mse:.4f} | MAE: {mae:.4f}")


oof_df = pd.DataFrame({
    "id": train["id"],
})

for idx, target in enumerate(targets):
    oof_df[target] = oof[:, idx]


[I 2025-07-14 08:43:32,431] A new study created in memory with name: GNNMolecularPredictor_optimization



=== Fold 1 / 5 ===

Parameter Search Configuration:
--------------------------------------------------

 Parameters being searched:
  • augmented_feature: ['maccs,morgan', 'maccs', 'morgan', None]
  • drop_ratio: [0.0, 0.75]
  • gnn_type: ['gin-virtual', 'gcn-virtual', 'gin', 'gcn']
  • graph_pooling: ['mean', 'sum', 'max']
  • hidden_size: [64, 512]
  • learning_rate: [1e-05, 0.01]
  • norm_layer: ['batch_norm', 'layer_norm', 'instance_norm', 'graph_norm', 'size_norm', 'pair_norm']
  • num_layer: [2, 8]
  • scheduler_factor: [0.1, 0.5]
  • weight_decay: [1e-08, 0.001]

 Fixed parameters (not being searched):
  • batch_size: 128
  • device: cuda:0
  • epochs: 10
  • evaluate_criterion: <function mean_absolute_error at 0x7f2dd2bc00d0>
  • evaluate_higher_better: False
  • evaluate_name: mae
  • fitting_epoch: 0
  • fitting_loss: []
  • grad_clip_value: None
  • loss_criterion: L1Loss()
  • num_task: 5
  • patience: 50
  • scheduler_patience: 5
  • task_type: regression
  • use_lr_sched

  0%|          | 0/1 [00:00<?, ?it/s]


Converting molecules to graphs: 100%|██████| 7179/7179 [00:13<00:00, 540.23it/s]

Converting molecules to graphs: 100%|██████| 1795/1795 [00:03<00:00, 540.42it/s]

ining:  98%|██████████▊| 56/57 [00:03<00:00, 16.53it/s, Epoch=0, Loss=0.8298]
                                                                             

Better result found at Epoch 0: Loss = 6.4456, mae = 21.5788


...

Trial 0: mae = 16.4826 (better than best = 16.4826)
Current parameters:
  gnn_type: gin-virtual
  norm_layer: pair_norm
  graph_pooling: max
  augmented_feature: ['morgan']
  num_layer: 2
  hidden_size: 340
  drop_ratio: 0.16802580393400884
  scheduler_factor: 0.4180168907883145
  learning_rate: 0.0003470705538770503
  weight_decay: 4.2219454157294834e-08
[I 2025-07-14 08:47:29,632] Trial 0 finished with value: 16.48255937794452 and parameters: {'gnn_type': 'gin-virtual', 'norm_layer': 'pair_norm', 'graph_pooling': 'max', 'augmented_feature': 'morgan', 'num_layer': 2, 'hidden_size': 340, 'drop_ratio': 0.16802580393400884, 'scheduler_factor': 0.4180168907883145, 'learning_rate': 0.0003470705538770503, 'weight_decay': 4.2219454157294834e-08}. Best is trial 0 with value: 16.48255937794452.

Optimization completed successfully:
Best mae: 16.4826


Predicting: 100%|██████████████████████████████| 15/15 [00:00<00:00, 104.56it/s]


post score is:  16.482559346535687

Best parameters:
  gnn_type: gin-virtual (type: categorical)
  norm_layer: pair_norm (type: categorical)
  graph_pooling: max (type: categorical)
  augmented_feature: ['morgan'] (type: categorical)
  num_layer: 2 (type: integer)
  hidden_size: 340 (type: integer)
  drop_ratio: 0.16802580393400884 (type: float)
  scheduler_factor: 0.4180168907883145 (type: float)
  learning_rate: 0.0003470705538770503 (type: log_float)
  weight_decay: 4.2219454157294834e-08 (type: log_float)

Optimization statistics:
  Number of completed trials: 1
  Number of pruned trials: 0
  Number of failed trials: 0


Predicting: 100%|████████████████████████████████| 1/1 [00:00<00:00, 178.88it/s]

Fold 5 target: Tg MSE: 11796.9424 | MAE: 78.0071
Fold 5 target: FFV MSE: 0.0124 | MAE: 0.0952
Fold 5 target: Tc MSE: 0.0034 | MAE: 0.0439
Fold 5 target: Density MSE: 0.2443 | MAE: 0.3795
Fold 5 target: Rg MSE: 40.2469 | MAE: 3.8870


In [86]:
if is_kaggle_notebook:
    submission.to_csv("submission.csv", index=False)
    print("✅ submission saved to submission.csv")
else:
    # CV 計算
    # oof_df = pd.DataFrame()
    # oof_df["id"] = train["id"]
    # for i_oof in oof_dfs:
    #     oof_df = oof_df.merge(i_oof, on="id", how="left")
    solution = train[["id"] + targets].copy()
    solution = solution.fillna(NULL_FOR_SUBMISSION)
    oof_df = oof_df.fillna(NULL_FOR_SUBMISSION)
    
    # 評価
    final_score = score(
        solution=solution,
        submission=oof_df,
    )
    print(f"\n📊 Final OOF Score (wMAE): {final_score:.6f}")



📊 Final OOF Score (wMAE): 0.160224


In [25]:
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

# print(train.shape)
# print(oof_df.shape)
# display(train.head())

# print(output_path)
# train.to_csv(output_path / "train.csv", index=False)
# oof_df.to_csv(output_path / "oof.csv", index=False)


(8974, 8)
(8974, 6)


,id,SMILES,Tg,FFV,Tc,Density,Rg,scaffold
0,0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN,c1ccccc1
1,1,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN,c1ccc(Cc2ccc(C3(c4ccc(Cc5ccccc5)cc4)CCCCC3)cc2...
2,2,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN,O=C1C(=Cc2ccccc2)CCCC1=Cc1ccc(Oc2ccc(S(=O)(=O)...
3,3,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN,c1ccc(-c2cc(-c3ccccc3)c(-c3ccccc3)c(-c3ccccc3)...
4,4,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN,O=C(Oc1ccccc1)c1cc(OCCCCCCCCCOCC2CCCN2c2ccccc2...


../outputs/exp015
